In [ ]:

import tensorflow as tf
import numpy as np 
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input

In [ ]:
train_dir = '/work/Mask_No_Mask/dataset'

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)


train_generator = train_datagen.flow_from_directory(
       train_dir,
        target_size=(224,224),
        batch_size=16,
        class_mode='categorical')

Found 434 images belonging to 5 classes.


## Multi-Class Image Classification (5 classes)

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D


# create the base pre-trained model
base_model = tf.keras.applications.ResNet101(weights='imagenet', include_top=False)

x = base_model.output

x = GlobalAveragePooling2D()(x)

x = Dense(1024, activation='relu')(x)

# and a logistic layer -- let's say we have 200 classes
predictions = Dense(5, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False




# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
# train the model on the new data for a few epochs
model.fit(train_generator, epochs=1)

28/28 [==============================] - 138s 5s/step - loss: 2.0514 - acc: 0.2522


In [ ]:
path = '/work/Mask_No_Mask/dataset/design_mask/joby_1616487809.jpg'

In [ ]:
img=image.load_img(path, target_size=(224, 224))
  
x=image.img_to_array(img)
x=np.expand_dims(x, axis=0)
images = np.vstack([x])

classes = model.predict(images, batch_size=1)
pred = classes[0].argmax()
pred

4

## Multi-Output Class: Classification + Regression 

In [ ]:
# Load the Resnet Model

image_size = 224

backbone = tf.keras.applications.ResNet101( input_tensor = Input(
    shape=(image_size, image_size, 3)),
    include_top=False,
    weights='imagenet'
    )



#### Loss, Weights, Metrics And Model Compilation

In [ ]:
losses = {
    "box_output": "mean_squared_error",
    "class_output": "categorical_crossentropy"
    }


# Here you can give more or less weightage to each loss.

# If you think that detection is harder then the classification then you can

# Try assinging it more weight
loss_weights = {
    "box_output": 1.0,
    "class_output": 1.0
    }

 

# For the class labels we want to know the Accuracy
# And for the bounding boxes we need to know the Mean squared error
metrics = {
    'class_output': 'accuracy',
    'box_output':  'mse'
    }

 
# Initialize Optimizer
opt = tf.keras.optimizers.SGD(lr = 1e-2, momentum = 0.9)
 


In [ ]:




def create_model(no_of_classes):

    # Freeze the whole model

    backbone.trainable = False

    # Start by taking the output feature maps from Resnet
    base_model_output = backbone.output

     

    # Convert to a single-dimensional vector by Global Average Pooling.
    flattened_output = GlobalAveragePooling2D()(base_model_output)

 

    # Create our Classification Head, final layer contains

    # Ouput units = no. classes
    class_prediction = Dense(256, activation="relu")(flattened_output)
    class_prediction = Dense(128, activation="relu")(class_prediction )
    class_prediction = Dropout(0.2)(class_prediction)
    class_prediction = Dense(64, activation="relu")(class_prediction)
    class_prediction = Dropout(0.2)(class_prediction )
    class_prediction = Dense(32, activation="relu")(class_prediction)
    class_prediction = Dense(no_of_classes, activation='softmax', name="class_output")(class_prediction)


    # Create Our Localization Head, final layer contains 4 nodes for x1,y1,x2,y2
    # Respectively.
    box_output = Dense(256, activation="relu")(flattened_output)
    box_output = Dense(128, activation="relu")(box_output)
    box_output = Dropout(0.2)(box_output )
    box_output = Dense(64, activation="relu")(box_output)
    box_output = Dropout(0.2)(box_output )
    box_output = Dense(32, activation="relu")(box_output)
    box_predictions = Dense(5, activation='sigmoid', name= "box_output")(box_output)

    # Now combine the two heads
    model = tf.keras.Model(inputs=backbone.input, outputs= [box_predictions,  class_prediction])

    return model

In [ ]:
model = create_model(5)

In [ ]:
# Compile the model with Adam optimizer
model.compile(optimizer = opt, loss = losses, loss_weights = loss_weights,
    metrics = metrics)

In [ ]:
# train the model on the new data for a few epochs
model.fit(train_generator, epochs=10)

Epoch 1/10
28/28 [==============================] - 228s 8s/step - loss: 1.9616 - box_output_loss: 0.2382 - class_output_loss: 1.7234 - box_output_mse: 0.2382 - class_output_accuracy: 0.1667
Epoch 2/10
28/28 [==============================] - 183s 6s/step - loss: 1.7796 - box_output_loss: 0.1690 - class_output_loss: 1.6107 - box_output_mse: 0.1690 - class_output_accuracy: 0.2842
Epoch 3/10
28/28 [==============================] - 172s 6s/step - loss: 1.7640 - box_output_loss: 0.1665 - class_output_loss: 1.5974 - box_output_mse: 0.1665 - class_output_accuracy: 0.2816
Epoch 4/10
28/28 [==============================] - 174s 6s/step - loss: 1.7667 - box_output_loss: 0.1616 - class_output_loss: 1.6052 - box_output_mse: 0.1616 - class_output_accuracy: 0.2673
Epoch 5/10
28/28 [==============================] - 175s 6s/step - loss: 1.7612 - box_output_loss: 0.1619 - class_output_loss: 1.5992 - box_output_mse: 0.1619 - class_output_accuracy: 0.2737
Epoch 6/10
28/28 [===========================

55/55 [==============================] - 101s 2s/step - loss: 1.8642 - box_output_loss: 0.1874 - class_output_loss: 1.6768 - box_output_mse: 0.1874 - class_output_accuracy: 0.2185


In [ ]:
from keras.preprocessing import image
import numpy as np

In [ ]:
path = '/work/Mask_No_Mask/dataset/design_mask/joby_1616487809.jpg'

In [ ]:
img=image.load_img(path, target_size=(224, 224))
  
x=image.img_to_array(img)
x=np.expand_dims(x, axis=0)
images = np.vstack([x])

classes = model.predict(images, batch_size=10)

In [ ]:
classes

[array([[0.1816431 , 0.16929132, 0.1952095 , 0.18487835, 0.36384985]],
       dtype=float32),
 array([[0.16636616, 0.15433702, 0.22577547, 0.10836096, 0.34516037]],
       dtype=float32)]

In [ ]:
 print(classes[0])

[[0.1816431  0.16929132 0.1952095  0.18487835 0.36384985]]


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9f1e7bdb-154a-4549-a646-7775bf1dc7a2' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>